In [ ]:
from llmtosql.model import WikiSQLModel
from llmtosql.trainer import Trainer
from llmtosql.dataloader import WikiSQLDataset
from llmtosql.utils.utils import plot_history, load_model, load_history
from matplotlib import pyplot as plt
import sys
import os
import json
import numpy as np
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
sys.path.insert(0, '..')

In [ ]:
model = WikiSQLModel(base_model_type='bert-base-cased', heads=(False, False, True), inference=False)
# try:
#     model = load_model(model, 'model_output/model.pth')
# except Exception as e:
#     print(f'No model loaded - {e}')

In [ ]:
train_set =  WikiSQLDataset(type='dev', model=model)

In [ ]:
train_set.maxcondsLength

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

loss = CrossEntropyLoss()
loader = DataLoader(train_set, batch_size=3)

In [ ]:
for data in loader:
    label = data['labels']
    break

In [ ]:
# val_set = WikiSQLDataset(type='dev')

In [ ]:
len(train_set.data)

In [ ]:
# datasets = (train_set, val_set)

In [ ]:
config = {
    'seed': 32,
    'scheduler': None,
    'optimizer': 'adam',
    'momentum': 0.9,
    'weight_decay': 0.0,
    'lr': 0.0005,
    'criterion': 'custom',
    'metric': 'accuracy',
    'pred_function': 'softmax',
    'model_dir': 'model_output',
    'backend': 'smddp'
}

In [ ]:
trainer = Trainer(model, datasets=train_set,
                  epochs=10, batch_size=32,
                  is_parallel=False,
                  **config)

In [ ]:
trainer.fit()

In [ ]:
trainer.save_history_('model_output')

In [ ]:
history = load_history('model_output')

In [ ]:
plot_history(history)

In [ ]:
model = WikiSQLModel(base_model_type='bert-base-uncased', attention_type='sqlnet', col_drop=True)
model = load_model(model, 'model_output/model.pth')
test_loss, test_accuracy = trainer.test(model, trainer.val_loader)

In [ ]:
print(f'Model accuracy on test: {test_accuracy}')